In [1]:
from kb_utils import *

### Creazione della KB

In [3]:
# creazione della kb
create_kb()

### Feature engineering tramite KB `facts.pl` e `clauses.pl`

In [4]:
# consulto della kb
prolog = Prolog()
prolog.consult('facts.pl')
prolog.consult('clauses.pl')

In [5]:
# derivazione del dataset 'movies_not_binned.csv' da 'movies_v2.csv'
df_movies = pd.read_csv('../dataset/movies_v2.csv')

# derivo il dataframe dei film con features non binnate
df_movies_not_binned = derive_movies_data(df_movies, prolog, binning=False)
df_movies_not_binned.to_csv('../dataset/movies_not_binned.csv', index=False)

In [6]:
# derivazione del dataset 'movies_binned.csv' da 'movies_v2.csv'
df_movies = pd.read_csv('../dataset/movies_v2.csv')

# derivo il dataframe dei film con features binnate
df_movies_binned = derive_movies_data(df_movies, prolog, binning=True)
df_movies_binned.to_csv('../dataset/movies_binned.csv', index=False)